In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from time import sleep
from random import randint, randrange
import os

import pandas as pd
import numpy as np

ua = UserAgent(verify_ssl=False)
header = {'User-Agent':str(ua.firefox)}

Initially, I used the General Election 2018 Dashboard website to fetch URLs for each constituency. 

In [ ]:
'''
r = requests.get('https://www.ecp.gov.pk/resultdashboard/ge2018.aspx',headers=header)
soup = BeautifulSoup(r.text, "lxml")

# From the dashboard page, fetch URLs for all the NA seats
na_urls = [i.find('a')['href'][2:].replace(' ','%20')\
           for i in soup.find_all(name='div',attrs={'class':'card card-hover'})\
           if i.find('a') != None]
'''

However, this page has been taken down for some reason so instead I use a list of constituency numbers and insert this into the URL that holds data for each individual constituency.

We leave out NA-60 and NA-103 because elections here were postponed.

In [ ]:
# Generate a list of constituency names to insert into the URLs
naSeats = ['NA-' + str(i) for i in np.arange(1,273) if i not in [60,103]]

In [ ]:
dataframe = []
count = 0
sleep_between_requests == False

for seat in naSeats:
    url = 'https://www.ecp.gov.pk/ConstResult.aspx?Const_Id=' + seat + '&type=NA&Election_ID=10070&Election=GENERAL%20ELECTION%2025%20JUL%202018'
    print('\n--- Fetching HTML for Seat # : ' + str(seat))
    r = requests.get(url,headers=header)
    
    if r.status_code == 200:
        print('--- HTML retrieved. Extracting Data')
        soup = BeautifulSoup(r.text, "lxml")
    else:
        print('*** Abort. HTML Status Code ' + str(r.status_code))
        break
    
    # Extract seat info and result status
    seatInfo = soup.find_all('span',{'id' : 'ContentPlaceHolder2_lblSubHeading'})[0].getText()
    seatName = seatInfo[seatInfo.find('(') + 1 : seatInfo.find(')')]
    seatStatus = seatInfo[seatInfo.find(')') + 1 : ].strip()
    
    # Extract voting statistics
    stats = soup.find_all('table')[0]
    registeredVoters = stats.find('span',{'id': 'ContentPlaceHolder1_lblRegVoters'}).getText()
    votesPolled = stats.find('span',{'id': 'ContentPlaceHolder1_lblVotesPolled'}).getText()
    validVotes = stats.find('span',{'id': 'ContentPlaceHolder1_lblValidVotes'}).getText()
    rejectedVotes = stats.find('span',{'id': 'ContentPlaceHolder1_lblRejVotes'}).getText()
    polledToRegisteredRatio = stats.find('span',{'id': 'ContentPlaceHolder1_lblTO'}).getText().replace('%','').strip()
    
    # Extract voting results
    voteCount = soup.find_all('table')[1]
    votingResults = []
    for i in voteCount.find_all('tr'):
        row = i.find_all('p')
        if len(row) != 0:
            candidateName = row[0].getText()
            candidateParty = row[1].getText()
            candidateVotes = row[2].getText()
            votingDict = {'candidateName' : candidateName,\
                          'candidateParty' : candidateParty,\
                          'candidateVotes' : int(candidateVotes)}
            votingResults.append(votingDict)
        
    data = {'seat' : seat,\
            'seatName' : seatName,\
            'seatStatus' : seatStatus,\
            'registeredVoters' : int(registeredVoters),\
            'votesPolled' : int(votesPolled),\
            'validVotes' : int(validVotes),\
            'rejectedVotes' : int(rejectedVotes),\
            'polledToRegRatio' : float(polledToRegisteredRatio) / 100,\
            'numberOfCandidates' : len(votingResults),\
            'votingResults' : votingResults,
            }
    
    dataframe.append(data)
    print('--- ' + seat + ' data addedd succesfully.')
    count += 1
    
    if sleep_between_requests == True:
        # Sleep for a few seconds before moving on to the next seat
        sleep_seconds = randint(2,10)
        print('--- Sleeping For : ', sleep_seconds, ' seconds.\n')
        sleep(sleep_seconds)
    
print('\n--- ' + str(count) + ' seats processed.')  
print('There should be 270 seats : ', len(dataframe) == 270)

The ECP website has voting results for NA-39 all wrong. For now, this is the only constituency that has faulty data on the website that I'm aware of but I'm pretty sure there are more. Let me know if you come across any.

In [ ]:
dataframe[38]['votingResults'] = [
    {'candidateName' : 'Muhammad Yaqoob Sheikh', 'candidateParty' : 'Pakistan Tehreek-e-Insaf', 'candidateVotes' : 5511},
    {'candidateName' : 'Alhaj Sardar Umar Farooq Khan', 'candidateParty' : 'Independent', 'candidateVotes' : 29},
    {'candidateName' : 'Asmat Ullah', 'candidateParty' : 'Independent', 'candidateVotes' : 98},
    {'candidateName' : 'Eithesham Ullah Khan', 'candidateParty' : 'Independent', 'candidateVotes' : 5},
    {'candidateName' : 'Faisal Karim Kundi', 'candidateParty' : 'Independent', 'candidateVotes' : 20},
    {'candidateName' : 'Fazl ur Rehman', 'candidateParty' : 'Muttahida Majlis-e-Amal Pakistan', 'candidateVotes' : 4076},
    {'candidateName' : 'Irfan Ullah Khan', 'candidateParty' : 'Independent', 'candidateVotes' : 20},
    {'candidateName' : 'Muhammad Aftab Inayat', 'candidateParty' : 'Independent', 'candidateVotes' : 105},
    {'candidateName' : 'Muhammad Akbar Khan', 'candidateParty' : 'Independent', 'candidateVotes' : 57},
    {'candidateName' : 'Muhammad Amir', 'candidateParty' : 'Independent', 'candidateVotes' : 12},
    {'candidateName' : 'Muhammad Maqbool', 'candidateParty' : 'Amun Taraqqi Party', 'candidateVotes' : 23},
    {'candidateName' : 'Nurang Khan', 'candidateParty' : 'Pakistan Peoples Party Parliamentarians', 'candidateVotes' : 292},
    {'candidateName' : 'Obaid Ur Rehman', 'candidateParty' : 'Independent', 'candidateVotes' : 32},
    {'candidateName' : 'Qaizar Khan', 'candidateParty' : 'Independent', 'candidateVotes' : 564},
    {'candidateName' : 'Waqar Ahmad Khan', 'candidateParty' : 'Independent', 'candidateVotes' : 98},
]

#### We will also retrieve voter participation rates, which is also bifurcated by gender.

In [ ]:
participationResult = []

r = requests.get('https://www.ecp.gov.pk/frmstats.aspx',headers=header)
soup = BeautifulSoup(r.text, "lxml")
rows = soup.find_all('tr')

for i in rows[1:]: # We don't look at the first row since these are just headers.
    row = i.find_all('td')
    participationResult.append(
        {
        'seat' : row[0].getText(),
        'femaleTurnout' : float(row[1].getText().replace(' %','')) / 100,
        'maleTurnout' : float(row[2].getText().replace(' %','')) / 100,
        'totalTurnout' : float(row[3].getText().replace(' %','')) / 100,
        }
    )
    
dfParticipation = pd.DataFrame(participationResult)

#### Store as a Pandas dataframe and store in a CSV

In [ ]:
df = pd.DataFrame(dataframe)
df = df.merge(dfParticipation,on='seat')
df.to_csv('../Data/Election_2018_NA_Results_Raw.csv',encoding='utf-8',index=False)

#### The voting results is a list of dictionaries contained in the dataframe above. For easier access, we will convert this to it's own dataframe and store it in a CSV as well.

In [ ]:
resultsOnlyDataframe = []
for i in dataframe:
    resultsDict = i['votingResults']
    for j in resultsDict:
        j['seat'] = i['seat']
        j['seatName'] = i['seatName']
        resultsOnlyDataframe.append(j)

In [ ]:
dfResultsOnly = pd.DataFrame(resultsOnlyDataframe)

In [ ]:
dfResultsOnly.to_csv('../Data/Election_2018_NA_Results_VotingOnly.csv',encoding='utf-8',index=False)

## Winners & Runner Ups

In [ ]:
# Adding margins from winners.

seat = ''
for row in dfResultsOnly.iterrows():
    if row[1]['seat'] == seat:
        margin = winnerVotes - row[1]['candidateVotes']
    else:
        winnerVotes = dfResultsOnly[dfResultsOnly['seat'] == row[1]['seat']][['candidateVotes']].max().values[0]
        margin = winnerVotes - row[1]['candidateVotes']
        seat = row[1]['seat']

    dfResultsOnly.loc[row[0],'marginFromWinner'] = int(margin)

#### Winners & Runner Ups DF & CSV

In [ ]:
winnerIdx = dfResultsOnly.groupby(by='seat',)[['candidateVotes']].idxmax()
winnerResults = dfResultsOnly.iloc[winnerIdx['candidateVotes'].values]

runnerUpIdx = dfResultsOnly.groupby(by='seat')['candidateVotes'].nlargest(2).reset_index().groupby('seat').last()['level_1'].values
runnerupResults = dfResultsOnly.iloc[runnerUpIdx]

winnerResults = winnerResults.drop('marginFromWinner',axis=1)
winnerResults = winnerResults.merge(runnerupResults[['seat','marginFromWinner']],on='seat').rename(columns={'marginFromWinner' : 'winMargin'})
winnerResults.to_csv('../Data/Election_2018_NA_winnerResults.csv',index=False,encoding='utf-8')
runnerupResults.to_csv('../Data/Election_2018_NA_runnerupResults.csv',index=False,encoding='utf-8')